In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
testloader= torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)
classes=('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 6 Layers : 3 conv layers and 3 fully connected layers !
        self.conv1 = nn.Conv2d(3, 8, 3,padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout2d(p=0.2)
        self.conv2 = nn.Conv2d(8, 16, 5,padding=2)
        self.conv3 = nn.Conv2d(16, 32, 5,padding=2)
        self.fc1 = nn.Linear(32*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x = x.view(-1, 32 *4* 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
learning_rate=[0.001,0.01]
model_store=[]

In [8]:
# net=Net()

# Create an instance of the model on CUDA
for LR in learning_rate:
    net = Net().cuda()

    lossvsiter=[]

    # To see if the model is on CUDA or not !
    if (next(net.parameters()).is_cuda) :
        print("The model is on CUDA")
    else :
        print("The model is on CPU")

    # Import the optimizers 
    import torch.optim as optim

    # Declare a loss function
    criterion = nn.CrossEntropyLoss()

    # Declare an optimizer
    optimizer = optim.Adam(net.parameters(),lr=LR)

    #No of iterations !
    iterations = 1


    for epoch in range(iterations):  # loop over the dataset multiple times

        # Reset the loss for the current epoch !
        running_loss = 0.0

        # Loop over all the mini-batches therea are 12500 mini batches of size 4 each !
        for i, data in enumerate(trainloader, 0):
            # get the inputs
            inputs, labels = data

            # wrap them in Variable & if possible make them cuda tensors
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

            # zero the parameter gradients for the current epoch
            optimizer.zero_grad()

            # forward + backward + optimize

            # forward
            outputs = net(inputs)
            loss = criterion(outputs, labels)

            # Calculate gradients of whatever variable set to req_gardients = True
            loss.backward()

            # Take one step of the gradient descent for this epoch ! 
            optimizer.step()

            # print statistics
            running_loss += loss.data[0]
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[Epoch :: %d, Mini Batch :: %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                lossvsiter.append(running_loss / 2000)
                running_loss = 0.0


    print('Finished Training')
    model_store.append(net)

The model is on CUDA
[Epoch :: 1, Mini Batch ::  2000] loss: 2.032
[Epoch :: 1, Mini Batch ::  4000] loss: 1.801
[Epoch :: 1, Mini Batch ::  6000] loss: 1.721
[Epoch :: 1, Mini Batch ::  8000] loss: 1.685
[Epoch :: 1, Mini Batch :: 10000] loss: 1.643
[Epoch :: 1, Mini Batch :: 12000] loss: 1.609
Finished Training
The model is on CUDA
[Epoch :: 1, Mini Batch ::  2000] loss: 2.307
[Epoch :: 1, Mini Batch ::  4000] loss: 2.307
[Epoch :: 1, Mini Batch ::  6000] loss: 2.306
[Epoch :: 1, Mini Batch ::  8000] loss: 2.307
[Epoch :: 1, Mini Batch :: 10000] loss: 2.350
[Epoch :: 1, Mini Batch :: 12000] loss: 2.306
Finished Training


In [10]:
model_store[1]

Net (
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (dropout): Dropout2d (p=0.2)
  (conv2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear (512 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)

In [16]:
import pickle

In [17]:
with open("PyTorch-Architecture-2.pkl","wb") as f:
    pickle.dump(lossvsiter,f)

In [12]:
correct = 0
total = 0

for data in testloader:
    images, labels = data
    images=images.cuda()
    labels=labels.cuda()
    try:
        outputs = model_store[1](Variable(images))
    except RuntimeError as re:
        print(outputs.is_cuda)
        print(str(re))
        sys.exit()
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 10 %


In [19]:
correct

5218

In [20]:
for i, data in enumerate(trainloader, 0):
    images, labels = data
    images=images.cuda()
    labels=labels.cuda()
    try:
        outputs = net(Variable(images))
    except RuntimeError as re:
        print(outputs.is_cuda)
        print(str(re))
        sys.exit()
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    if (i%1000) == 0:
        print(i)

print('Accuracy of the network on the 50000 trained images: %d %%' % (
    100 * correct / total))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
Accuracy of the network on the 50000 trained images: 53 %


In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    images=images.cuda()
    labels=labels.cuda()
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
lossvsiter

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
plt.plot(lossvsiter)
plt.ylabel('loss')
plt.xlabel('every 2000th mini-batch')
plt.show()

In [ ]:
# Lets save the model !!

In [ ]:
 torch.save(net.state_dict(),"./models/archi2")